In [3]:
!pip3 install --upgrade transformers optimum
!pip3 install --upgrade auto-gptq


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.9/409.9 kB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 64.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [5]:
model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
# To use a different branch, change revision
# For example: revision="gptq-4bit-32g-actorder_True"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4225: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [9]:
example = '''[INST]Here is an example of medical report/findings from a medical image generated from keywords: keywords: {
      "fracture_type": {
        "category": "long bone",
        "sub_category": "tebia",
        "fracture_subtype": "extra articular",
        "specific_subtype": "spiral"
      }
      report : "Clinical History: The patient presented with localized pain and tenderness in the left lower extremity following trauma.Findings: Imaging of the left lower extremity was performed. The study reveals a fracture involving the proximal aspect of the tibia, classified as an extra-articular fracture. The fracture pattern is characterized as a spiral fracture. These findings are consistent with a spiral extra-articular fracture of the proximal tibia.Impressions: Further clinical correlation and management, including potential orthopedic intervention, are recommended. Consultation with an orthopedic specialist for further evaluation and management planning is advised. Consideration for additional imaging studies, such as CT or MRI, if warranted for further characterization of the fracture pattern and assessment of soft tissue involvement is also recommended. Initiation of appropriate pain management and immobilization measures as indicated is essential.Conclusion: The imaging findings demonstrate a spiral extra-articular fracture of the proximal tibia. Clinical correlation and orthopedic consultation are advised for further management.[/INST]"
      [INST] Similarly,based on the given example, generate a radiological medical report for the following keywords. [/INST]'''

In [16]:
keywords = '''fracture_type": {
        "category": "long bone",
        "sub_category": "femur",
        "fracture_subtype": "extra articular",
        "specific_subtype": "comminuted"
      }'''

In [12]:
prompt = f"<s>[INST] {example} : {keywords} [/INST]"

In [17]:
# Inference
input_ids = tokenizer(prompt, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.6, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
input_length = input_ids.size(1)
generated_tokens = output[:, input_length:]
generated =  tokenizer.decode(generated_tokens[0], skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [18]:
print(generated)

Clinical History: The patient presented with complaints of severe pain and swelling in the right upper extremity following a traumatic event.

Findings: Radiographic examination of the right upper extremity was conducted. The imaging study reveals a fracture in the distal portion of the femur, categorized as an extra-articular fracture. The fracture pattern is identified as comminuted. These findings are consistent with a comminuted extra-articular fracture of the distal femur.

Impressions: Further clinical evaluation and management, including potential orthopedic intervention, are recommended. Consultation with an orthopedic specialist for further evaluation and management planning is advised. Consideration for additional imaging studies, such as CT or MRI, if warranted for further characterization of the fracture pattern and assessment of soft tissue involvement is also recommended. Initiation of appropriate pain management and immobilization measures as indicated is essential.

Con